In [2]:
import pandas as pd
from fhir.resources.medicationadministration import MedicationAdministration
from fhir.resources.identifier import Identifier
from fhir.resources.period import Period
from fhir.resources.coding import Coding
from fhir.resources.codeableconcept import CodeableConcept
from datetime import datetime
import json
#import simplejson as json
import math
import fhir.resources
print(fhir.resources.__version__)
df = pd.read_csv('sample_health_data_v2/medication_administrations.tsv', sep='\t')
df

6.5.0


,id,status,effective,patient_id,encounter_id,medication_code,medication_system,medication_display
0,med_admin_1,Completed,10/05/2019 02:48:13 PM,pat_9,enc_78,1659131,http://www.nlm.nih.gov/research/umls/rxnorm,piperacillin 2000 MG / tazobactam 250 MG Injec...
1,med_admin_2,Completed,10/05/2019 02:48:13 PM,pat_9,enc_78,1807510,http://www.nlm.nih.gov/research/umls/rxnorm,150 ML vancomycin 5 MG/ML Injection
2,med_admin_3,Completed,11/27/2017 10:22:31 PM,pat_8,enc_69,1000126,http://www.nlm.nih.gov/research/umls/rxnorm,1 ML medroxyprogesterone acetate 150 MG/ML Inj...
3,med_admin_4,Completed,12/18/2013 10:04:00 PM,pat_8,enc_68,1000126,http://www.nlm.nih.gov/research/umls/rxnorm,1 ML medroxyPROGESTERone acetate 150 MG/ML Inj...
4,med_admin_5,Completed,10/29/2019 09:35:07 PM,pat_2,enc_12,389221,http://www.nlm.nih.gov/research/umls/rxnorm,Etonogestrel 68 MG Drug Implant
5,med_admin_6,Completed,03/06/2016 08:35:07 PM,pat_2,enc_11,389221,http://www.nlm.nih.gov/research/umls/rxnorm,Etonogestrel 68 MG Drug Implant
6,med_admin_7,Completed,09/01/2014 04:03:29 PM,pat_5,enc_38,312617,http://www.nlm.nih.gov/research/umls/rxnorm,predniSONE 5 MG Oral Tablet


In [4]:
with open('test/medication_administrations.ndjson', 'r') as file:
    dataa = [json.loads(line) for line in file]


'''
{'resourceType': 'MedicationAdministration',
  'identifier': [{'system': 'UPMC Custom Medication Administration ID',
    'value': 'med_admin_1'}],
  'status': 'Completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '1659131',
     'display': 'piperacillin 2000 MG / tazobactam 250 MG Injection'}]},
  'subject': {'reference': 'pat_9'},
  'effectiveDateTime': '2019-10-05T14:48:13-05:00'}
'''
dataa[0:2]



[{'resourceType': 'MedicationAdministration',
  'identifier': [{'system': 'UPMC Custom Medication Administration ID',
    'value': 'med_admin_1'}],
  'status': 'completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '1659131',
     'display': 'piperacillin 2000 MG / tazobactam 250 MG Injection'}],
   'text': 'piperacillin 2000 MG / tazobactam 250 MG Injection'},
  'subject': {'reference': 'pat_9'},
  'context': {'reference': 'enc_78'},
  'effectiveDateTime': '2019-10-05T14:48:13-04:00'},
 {'resourceType': 'MedicationAdministration',
  'identifier': [{'system': 'UPMC Custom Medication Administration ID',
    'value': 'med_admin_2'}],
  'status': 'completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '1807510',
     'display': '150 ML vancomycin 5 MG/ML Injection'}],
   'text': '150 ML vancomycin 5 MG/ML Injection'},
  'subject': {'reference': 'pa

In [22]:
import pandas as pd
from fhir.resources.observation import Observation
from fhir.resources.period import Period
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.reference import Reference
import json
from datetime import datetime
import re
from fhir.resources.identifier import Identifier
from datetime import datetime, timedelta, timezone
from fhir.resources.quantity import Quantity
import decimal

def convert_decimals_to_float(obj):
    """Recursively converts Decimal values in a dictionary or list to floats."""
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = convert_decimals_to_float(value)
    elif isinstance(obj, list):
        for i in range(len(obj)):
            obj[i] = convert_decimals_to_float(obj[i])
    elif isinstance(obj, decimal.Decimal):  # Convert Decimal to float
        return float(obj)
    return obj

def parse_date(date_str):
    try:
        # Parse date and time from the format MM/DD/YYYY HH:MM:SS AM/PM
        parsed_date = datetime.strptime(date_str, "%m/%d/%Y %I:%M:%S %p")
        # Set the timezone offset to -05:00
        offset = timezone(timedelta(hours=-5))
        # Apply the timezone offset
        parsed_date_with_offset = parsed_date.replace(tzinfo=offset)
        # Convert to ISO format (YYYY-MM-DDTHH:MM:SS±HH:MM)
        return parsed_date_with_offset.isoformat()
    except ValueError:
        print(f"Invalid date format for '{date_str}'. Expected format: MM/DD/YYYY HH:MM:SS AM/PM.")
        return None
        
def convert_dates(record):
    if isinstance(record, dict):
        for key, value in record.items():
            if isinstance(value, datetime):  # If the value is a datetime object
                # Convert it to the desired format with timezone offset
                record[key] = convert_datetime_to_iso(value)
            elif isinstance(value, list):  # If the value is a list of items
                for item in value:
                    convert_dates(item)  # Recursively handle nested dictionaries
            else:
                convert_dates(value)  # Recursively handle nested dictionaries in the value
    elif isinstance(record, list):  # Handle lists outside of dicts
        for item in record:
            convert_dates(item)  # Recursively handle items in the list
    return record

def convert_datetime_to_iso(date_obj):
    # Assuming the input datetime is naive, and you want to apply a timezone of -05:00
    offset = timezone(timedelta(hours=-5))
    # Convert naive datetime to timezone-aware datetime
    if date_obj.tzinfo is None:
        date_obj = date_obj.replace(tzinfo=offset)
    # Return the ISO format with timezone offset
    return date_obj.isoformat()

def save_to_ndjson(data, output_file):
    with open(output_file, 'w') as f:
        for record in data:
            # Convert dates before writing
            record = convert_dates(record)
            record = convert_decimals_to_float(record)
            # Convert each OrderedDict to JSON and write it to the file
            f.write(json.dumps(record) + '\n')

def set_identifiers(medadmin, row):
    identifier = Identifier(system = 'UPMC Custom Medication Administration ID',
            value = row['id'])
    
    medadmin.identifier = [identifier]
def set_observation_component(encounter):
    # Create the Coding object
    class_coding = Coding(
        system="http://terminology.hl7.org/CodeSystem/v3-ActCode",
        code="IMP"
    )
    
    # Wrap the Coding in a CodeableConcept
    class_codeable_concept = CodeableConcept(coding=[class_coding])
    
    # Assign the CodeableConcept to class_fhir
    encounter.class_fhir = [class_codeable_concept]
def set_observation_subject(observation, row):
    subject=Reference(reference=f"{row['patient_id']}")
    observation.subject = subject
def set_observation_encounter(observation, row):
    subject=Reference(reference=f"{row['encounter_id']}")
    observation.encounter = subject
def create_medadmin_datetime(row):
    return parse_date(row['effective'])


def create_medadmin_codeableconcept(row):
    """Creates a CodeableConcept for the Observation code."""
    return CodeableConcept(
        coding=[Coding(system=row['medication_system'], code=row['medication_code'], display=row['medication_display'])]
        ,text=row['medication_display'])
def create_medadmin_status(row):
    """something!"""
    return row['status']
def tsv_to_fhir_observations(tsv_file):
    df = pd.read_csv(tsv_file, sep='\t')
    medadmins = []

    for _, row in df.iterrows():
        print(create_medadmin_status(row))
        medadmin = MedicationAdministration(resourceType="MedicationAdministration", 
        status=create_medadmin_status(row), effectiveDateTime=create_medadmin_datetime(row),
        medicationCodeableConcept=create_medadmin_codeableconcept(row), subject=Reference(reference=f"{row['patient_id']}"),
                                           context=Reference(reference=f"{row['encounter_id']}"))
        set_identifiers(medadmin, row)
        #set_observation_component(observation, row)
        #set_observation_datetime(observation, row)
        #set_observation_subject(observation, row)
        #set_observation_encounter(observation, row)
        #set_observation_category(observation, row)
        medadmins.append(medadmin.dict())
    
    return medadmins
# Example usage
tsv_file = 'sample_health_data_v2/medication_administrations.tsv'
output_file = 'medication_administration.ndjson'
observations = tsv_to_fhir_observations(tsv_file)
'''
def print_types(data, level=0):
    """Recursively prints the type of each value in a dictionary or list."""
    indent = "  " * level  # Indentation for nested structures
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}{key}: {type(value).__name__}")
            print_types(value, level + 1)  # Recursive call for nested dict
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{indent}[{index}]: {type(item).__name__}")
            print_types(item, level + 1)  # Recursive call for nested list
    else:
        print(f"{indent}{data}: {type(data).__name__}")

# Usage
# Assuming `observation` is your Observation resource object
print("Types of each value in Observation resources:")

print_types(observations[0]) 
'''
# Save to NDJSON
save_to_ndjson(observations, output_file)

Completed
Completed
Completed
Completed
Completed
Completed
Completed


In [23]:
with open('medication_administration.ndjson', 'r') as file:
    data = [json.loads(line) for line in file]

# `data` is now a list of dictionaries (one for each JSON object in the file)
data[0:3]

[{'resourceType': 'MedicationAdministration',
  'identifier': [{'system': 'UPMC Custom Medication Administration ID',
    'value': 'med_admin_1'}],
  'status': 'Completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '1659131',
     'display': 'piperacillin 2000 MG / tazobactam 250 MG Injection'}],
   'text': 'piperacillin 2000 MG / tazobactam 250 MG Injection'},
  'subject': {'reference': 'pat_9'},
  'context': {'reference': 'enc_78'},
  'effectiveDateTime': '2019-10-05T14:48:13-05:00'},
 {'resourceType': 'MedicationAdministration',
  'identifier': [{'system': 'UPMC Custom Medication Administration ID',
    'value': 'med_admin_2'}],
  'status': 'Completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '1807510',
     'display': '150 ML vancomycin 5 MG/ML Injection'}],
   'text': '150 ML vancomycin 5 MG/ML Injection'},
  'subject': {'reference': 'pa

In [25]:
import json

def compare_dicts(dict1, dict2, path=""):
    """Recursively compare two dictionaries and return a list of differences."""
    differences = []
    
    # Get all keys in both dictionaries
    all_keys = set(dict1.keys()).union(set(dict2.keys()))
    
    for key in all_keys:
        # Full path for the current key
        current_path = f"{path}.{key}" if path else key

        if key not in dict1:
            differences.append({"path": current_path, "file1": None, "file2": dict2[key]})
        elif key not in dict2:
            differences.append({"path": current_path, "file1": dict1[key], "file2": None})
        else:
            # Check if the values are both dictionaries for nested comparison
            if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
                differences.extend(compare_dicts(dict1[key], dict2[key], path=current_path))
            # Check if the values are different
            elif dict1[key] != dict2[key]:
                differences.append({"path": current_path, "file1": dict1[key], "file2": dict2[key]})
    
    return differences

def compare_ndjson_files(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

        # Check if the number of lines match
        if len(lines1) != len(lines2):
            return False, "The NDJSON files have a different number of lines."

        all_differences = []

        # Compare line by line
        for i, (line1, line2) in enumerate(zip(lines1, lines2)):
            json_obj1 = json.loads(line1)
            json_obj2 = json.loads(line2)

            # Pop out 'status' from json_obj1 before comparison
            #json_obj1.pop('status', None)

            # Compare the JSON objects
            differences = compare_dicts(json_obj1, json_obj2)
            if differences:
                all_differences.append({
                    "line": i + 1,  # Line number (1-based index)
                    "differences": differences
                })

        return True if not all_differences else False, all_differences

def print_differences(differences):
    """Print the differences in a readable format."""
    if not differences:
        print("The NDJSON files are the same.")
    else:
        for diff in differences:
            print(f"Difference found in line {diff['line']}:")
            for item in diff["differences"]:
                print(f"- Path: {item['path']}")
                print(f"  File 1: {json.dumps(item['file1'], indent=2)}")
                print(f"  File 2: {json.dumps(item['file2'], indent=2)}")
                print("-" * 40)

# Example usage
file1 = 'medication_administration.ndjson'
file2 = 'test/medication_administrations.ndjson'

are_same, differences = compare_ndjson_files(file1, file2)

if are_same:
    print("The NDJSON files are the same.")
else:
    print("The NDJSON files are different:")
    print_differences(differences)


The NDJSON files are different:
Difference found in line 1:
- Path: status
  File 1: "Completed"
  File 2: "completed"
----------------------------------------
- Path: effectiveDateTime
  File 1: "2019-10-05T14:48:13-05:00"
  File 2: "2019-10-05T14:48:13-04:00"
----------------------------------------
Difference found in line 2:
- Path: status
  File 1: "Completed"
  File 2: "completed"
----------------------------------------
- Path: effectiveDateTime
  File 1: "2019-10-05T14:48:13-05:00"
  File 2: "2019-10-05T14:48:13-04:00"
----------------------------------------
Difference found in line 3:
- Path: status
  File 1: "Completed"
  File 2: "completed"
----------------------------------------
Difference found in line 4:
- Path: status
  File 1: "Completed"
  File 2: "completed"
----------------------------------------
Difference found in line 5:
- Path: status
  File 1: "Completed"
  File 2: "completed"
----------------------------------------
- Path: effectiveDateTime
  File 1: "2019-